In [ ]:
url = "https://eoimages.gsfc.nasa.gov/images/imagerecords/80000/80948/dolansprings_oli_2013088_canyon_lrg.jpg"

In [ ]:
url = "https://www.esa.int/var/esa/storage/images/esa_multimedia/images/2020/07/solar_orbiter_s_first_view_of_the_sun2/22133123-1-eng-GB/Solar_Orbiter_s_first_view_of_the_Sun.png"